# FRED-T5

In [1]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
device='cuda'
model.to(device)

In [4]:
lm_text='<LM>Принялся Кутузов рассказывать свою историю как он сюда попал. Началось'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
# input_ids=torch.tensor([tokenizer.encode(lm_text)])
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True)
print(tokenizer.decode(outputs[0][1:]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 с того, что он был в армии, служил в артиллерии.</s>


In [11]:
text = """
Ответь на вопрос с заданным контекстом, пожалуйста.

Вопрос: "Как быстро растут подмышки у человека"

Контекст: "Люди, как правило, имеют обильный рост волос. В том числе, подмышки то же растут очень быстро, что странно. Зафиксированная средняя скорость роста 5мм/год"

"""
input_ids=torch.tensor([tokenizer.encode(text)]).to(device)
# input_ids=torch.tensor([tokenizer.encode(lm_text)])
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][1:]))


Ответ: "Подмышки растут очень быстро, что странно. Зафиксированная средняя скорость


In [12]:
outputs[0][1:]

tensor([  203, 29153,    30,   458,  7148,   918,  2300, 20322,   939,  2156,
           16,   374,  7322,    18,   999, 10370, 11049, 22434,  7872],
       device='cuda:0')

In [18]:
text = """Отвечайте на вопросы. Кто был лидером СССР? Ответьте подробно исходя из этого контекста:
"СССР был образован на территории, которую к 1917 году занимала Российская империя без Финляндии, части Польского царства и некоторых других территорий.Перейти к разделу «#География СССР» Советская власть на этой территории установилась в результате Октябрьской революции 1917 года и последовавшей за ней Гражданской войны. 30 декабря 1922 года РСФСР, Украинская ССР, Белорусская ССР и Закавказская СФСР объединились в одно государство — Союз ССР с едиными органами политической власти со столицей в Москве, с сохранением де-юре за каждой союзной республикой права свободного выхода из Союза[5].Перейти к разделу «#История СССР»
22 июня 1941 года Германия при поддержке союзников напала на СССР. Началась Великая Отечественная война, после победы в которой СССР стал, наряду с США, сверхдержавой[6][7][8][9]. Советский Союз доминировал в мировой системе социализма, а также был соучредителем ООН, постоянным членом Совета Безопасности ООН с правом ветоПерейти к разделу «СССР во Второй мировой войне (1941—1945)».
Согласно положениям Конституции 1977 года СССР провозглашался единым союзным многонациональным[10] социалистическим[11] государствомПерейти к разделу «#Политическая система и идеология». Союзные республики считались суверенными государствами[12]. Порядок выхода республик из Союза с 1990 года регулировал специальный закон. Союзная республика имела право вступать в отношения с иностранными государствами, заключать с ними договоры и обмениваться дипломатическими и консульскими представителями, участвовать в деятельности международных организаций[13]. Среди 50 стран-основателей ООН, наряду с СССР, были и 2 его союзные республики: БССР и УССРПерейти к разделу «#Административно-территориальное деление СССР».
Из-за низкой эффективности экономической системы, сильной зависимости от цен на энергоресурсы, существенных расходов на гонку вооружений, массового дефицита товаров, межнациональных противоречий и других проблем во второй половине 1980-х годов в СССР произошёл экономический и политический кризис. Обострилось внутреннее политическое противостояние. Попытки реформирования советской системы (демократизация, переход к рыночной экономике и многопартийности) не помогли решить накопившиеся противоречия. В 1988—1991 годах произошёл ряд законодательных конфликтов между союзным центром и союзными республикамиПерейти к разделу «Перестройка (1985—1991)». Последствия всех этих событий привели к распаду СССР (см. Причины распада).
17 марта 1991 года состоялся Всесоюзный референдум о сохранении СССР, на котором 77,85 % граждан советских республик, принявших участие в референдуме, высказались за сохранение Союза как обновлённой федерации равноправных социалистических суверенных республик[~ 10]. Попытка государственного переворота ГКЧП в августе 1991 года вызвала массовые народные демонстрации по всему СССР в защиту демократического курса Перестройки и законно избранной в 1989 году государственной власти — Съезда народных депутатов СССР. Апогеем противостояния путчистам считаются события у Белого дома в Москве. В результате попытка захвата власти ГКЧП провалилась. 8 декабря 1991 года тремя государствами — учредителями Союза были подписаны Соглашения о прекращении существования СССР и создании СНГ[14]. 26 декабря 1991 года Совет Республик Верховного Совета СССР принял декларацию о прекращении существования СССР[15].
Российская Федерация[~ 11] была признана де-факто[16] государством-продолжателем СССР в международно-правовых отношениях[~ 12] и заняла его место в Совете Безопасности ООН[~ 13].Перейти к разделу «#Распад Советского Союза (1990—1991)» В то же время, согласно действующему законодательству Украины, последняя является преемницей прав и обязательств УССР и международных договоров СССР[17]. Между государствами существует незакрытый вопрос о государственном долге СССР[18]. Территория, которую ранее занимал СССР, именуется «постсоветским пространством». " """
input_ids=torch.tensor([tokenizer.encode(text)]).to(device)
# input_ids=torch.tensor([tokenizer.encode(lm_text)])
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][1:]))


"В СССР существовала единая система управления, которая была основана на принципах социалистического централизма


# E5-LARGE

In [21]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: Сколько сейчас стоит вейп с яблоком?',
               "passage: Check out the chart below to see how much protein you should be eating each day.",
               "passage: Вчера был на рынке вейпов, там они стоят около 1500 рублей"]



In [22]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

batch_dict

{'input_ids': tensor([[     0,     41,   1294,     12,   3642,   5045,  21308,   5608,     10,
         117776,  73203,      2,      1,      1,      1,      1,      1,      1,
              1,      1,      1],
        [     0,     41,   1294,     12, 191405,  12504,  18510,     49,   2519,
           1078,    135,    784,  76591,    419,     32,      2,      1,      1,
              1,      1,      1],
        [     0,  46692,     12,  38679,   1810,     70, 116287,  35064,     47,
           1957,   3642,   5045,  21308,    398,   5608,    186, 118992,  12638,
           5155,      5,      2],
        [     0,  46692,     12, 162924,   4387,     29,  68096,     49,   2519,
          27383,      4,   3926,   2792, 123618,   9953,  17551,  28403,      2,
              1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
outputs = model(**batch_dict)
outputs

In [25]:

embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
embeddings

tensor([[ 0.0288, -0.0159, -0.0375,  ..., -0.0203, -0.0491,  0.0263],
        [ 0.0071, -0.0181, -0.0085,  ...,  0.0050, -0.0659, -0.0058],
        [ 0.0144, -0.0148, -0.0208,  ..., -0.0258, -0.0240,  0.0014],
        [ 0.0062, -0.0285, -0.0352,  ...,  0.0144, -0.0495,  0.0215]],
       grad_fn=<DivBackward0>)

In [ ]:
!pip install sentence_transformers

In [31]:
scores = (embeddings[:2] @ embeddings[2:].T) * 100
scores

tensor([[86.7220, 73.2169],
        [67.5268, 83.2351]], grad_fn=<MulBackward0>)

In [ ]:
from sentence_transformers import SentenceTransformer

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'how much protein should a female eat'),
    get_detailed_instruct(task, '南瓜的家常做法')
]
# No need to add instruction for retrieval documents
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"
]
input_texts = queries + documents

model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

embeddings = model.encode(input_texts, convert_to_tensor=True, normalize_embeddings=True)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())
# [[91.92853546142578, 67.5802993774414], [70.38143157958984, 92.13307189941406]]


In [93]:
input_texts

['Instruct: Given a web search query, retrieve relevant passages that answer the query\nQuery: how much protein should a female eat',
 'Instruct: Given a web search query, retrieve relevant passages that answer the query\nQuery: 南瓜的家常做法',
 "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
 '1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅']

In [94]:
embeddings

tensor([[ 0.0202,  0.0112, -0.0451,  ..., -0.0266, -0.0423,  0.0389],
        [ 0.0486,  0.0594,  0.0040,  ..., -0.0214, -0.0263,  0.0198],
        [ 0.0327,  0.0041, -0.0503,  ..., -0.0388, -0.0127,  0.0501],
        [ 0.0477,  0.0566,  0.0092,  ..., -0.0110, -0.0086,  0.0239]],
       device='cuda:0')

In [41]:
scores

tensor([[86.7220, 73.2169],
        [67.5268, 83.2351]], grad_fn=<MulBackward0>)

In [63]:
!curl -O https://www.cbr.ru/Crosscut/LawActs/File/7690

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  254k  100  254k    0     0   227k      0  0:00:01  0:00:01 --:--:--  227k


In [47]:
!pip install pdfreader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.7 MB/s eta 0:00:00
  Created wheel for pdfreader: filename=pdfreader-0.1.12-py3-none-any.whl size=134538 sha256=7f77663a4eddfb92a96737c79207791ad859f5c79252ac4d7793eba74c9394f2
  Stored in directory: /root/.cache/pip/wheels/92/7d/f1/8b8a503d8b444a82e4c3c8b1230c02316c7579b90542de4495
Successfully built pdfreader


In [48]:
from pdfreader import PDFDocument, SimplePDFViewer

In [69]:
fd = open("7690", "rb")
doc = SimplePDFViewer(fd)

In [65]:
doc.metadata

{'Author': '',
 'CreationDate': datetime.datetime(2024, 2, 29, 14, 49, 43, tzinfo=tzoffset(None, 10800)),
 'Creator': 'Aspose Ltd.',
 'ModDate': datetime.datetime(2024, 2, 29, 16, 51, 57, tzinfo=tzoffset(None, 10800)),
 'Producer': 'Aspose.PDF for .NET 21.5.0',
 'Subject': '',
 'Title': ''}

In [72]:
doc.render()
text = "".join(doc.canvas.strings)

In [73]:
text

'    ЦЕНТРАЛЬНЫЙ БАНК РОССИЙСКОЙ ФЕДЕРАЦИИ (БАНК РОССИИ) 107016, Москва, ул. Неглинная, д. 12, к. В www.cbr.ru           тел.: (499) 300-30-00, 8 (800) 300-30-00   От [REGNUMDATESTAMP]  Информационное письмо Банка России о рисках совершения операций с цифровыми валютами и рекомендациях финансовым организациям исключить предложение услуг, предусматривающих совершение таких операций       Кредитным организациям  Некредитным финансовым организациям     Результаты мониторинга состояния финансового рынка Российской Федерации и осуществления Банком России надзорной деятельности свидетельствуют о сохранении интереса к рынку цифровых валют  среди клиентов финансовых организаций, поднадзорных Банку России  (далее – финансовые организации).   Ранее Банк России неоднократно отмечал, что распространение цифровых валют в Российской Федерации сопряжено с высокими рисками  для благосостояния российских граждан и стабильности финансовой   системы, а также рисками вовлечения клиентов финансовых организ

In [74]:
doc.render()
text = "".join(doc.canvas.strings)

In [75]:
text

'    ЦЕНТРАЛЬНЫЙ БАНК РОССИЙСКОЙ ФЕДЕРАЦИИ (БАНК РОССИИ) 107016, Москва, ул. Неглинная, д. 12, к. В www.cbr.ru           тел.: (499) 300-30-00, 8 (800) 300-30-00   От [REGNUMDATESTAMP]  Информационное письмо Банка России о рисках совершения операций с цифровыми валютами и рекомендациях финансовым организациям исключить предложение услуг, предусматривающих совершение таких операций       Кредитным организациям  Некредитным финансовым организациям     Результаты мониторинга состояния финансового рынка Российской Федерации и осуществления Банком России надзорной деятельности свидетельствуют о сохранении интереса к рынку цифровых валют  среди клиентов финансовых организаций, поднадзорных Банку России  (далее – финансовые организации).   Ранее Банк России неоднократно отмечал, что распространение цифровых валют в Российской Федерации сопряжено с высокими рисками  для благосостояния российских граждан и стабильности финансовой   системы, а также рисками вовлечения клиентов финансовых организ

In [66]:
for canvas in doc:
  page_images = canvas.images
  page_forms = canvas.forms
  page_text = canvas.text_content
  page_inline_images = canvas.inline_images
  page_strings = canvas.strings
  print(page_strings)

[' ', ' ', ' ', ' ', 'ЦЕНТР', 'А', 'ЛЬНЫЙ', ' ', 'БАНК', ' ', 'РОССИЙСКОЙ', ' ', 'ФЕДЕ', 'РА', 'Ц', 'ИИ', ' ', '(БАНК', ' ', 'РОССИИ)', ' ', '107016, ', 'Москва,', ' ', 'ул.', ' ', 'Неглинн', 'а', 'я,', ' ', 'д.', ' 12, ', 'к.', ' ', 'В', ' ', 'www', '.', 'cbr', '.', 'ru', ' ', '          ', 'тел.:', ' (499) 300-30-00, 8 (800) 300-30-00 ', ' ', ' ', 'От', ' ', '[REGNUM', 'DATESTAMP]', ' ', ' ', 'Информационн', 'ое', ' ', 'п', 'исьм', 'о', ' ', 'Б', 'а', 'н', 'к', 'а', ' ', 'России', ' ', 'о', ' ', 'рисках', ' ', 'соверше', 'н', 'ия', ' ', 'операций', ' ', 'с', ' ', 'цифровыми', ' ', 'валютами', ' ', 'и', ' ', 'рекоме', 'н', 'д', 'аци', 'я', 'х', ' ', 'фи', 'нансовым', ' ', 'организациям', ' ', 'исключить', ' ', 'предложение', ' ', 'услуг,', ' ', 'предусм', 'атрив', 'а', 'ю', 'щих', ' ', 'соверше', 'н', 'ие', ' ', 'так', 'и', 'х', ' ', 'оп', 'ера', 'ц', 'ий', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'Кредитным', ' ', 'организациям', ' ', ' ', 'Некред', 'и', 'тным', ' ', 'ф', 'и', 'нансовым', 

In [85]:
import subprocess
import os

class DocumentHolder:
    try:
        os.mkdir("docs")
    except:
        pass

    N = len(os.listdir("docs"))

    def __init__(self, url) -> None:
        self.url = url
        self.text = {}
        self.load_doc()
        self.read_file()

    def load_doc(self):
        try:
            subprocess.call(f"curl -o docs/{self.N}.pdf {self.url}", shell=True, executable="/bin/bash")
            self.filename = f"{self.N}.pdf"
            self.N += 1
        except Exception as ex:
            print(ex)

    def read_file(self):
        self.fd = open("docs/" + self.filename,  "rb")
        self.viewer = SimplePDFViewer(self.fd)

        cnt = 1
        for canvas in self.viewer:
            self.text[cnt] = "".join(canvas.strings)
            cnt += 1

doc = DocumentHolder("https://www.cbr.ru/Crosscut/LawActs/File/7690")

In [87]:
doc.text

{1: '    ЦЕНТРАЛЬНЫЙ БАНК РОССИЙСКОЙ ФЕДЕРАЦИИ (БАНК РОССИИ) 107016, Москва, ул. Неглинная, д. 12, к. В www.cbr.ru           тел.: (499) 300-30-00, 8 (800) 300-30-00   От [REGNUMDATESTAMP]  Информационное письмо Банка России о рисках совершения операций с цифровыми валютами и рекомендациях финансовым организациям исключить предложение услуг, предусматривающих совершение таких операций       Кредитным организациям  Некредитным финансовым организациям     Результаты мониторинга состояния финансового рынка Российской Федерации и осуществления Банком России надзорной деятельности свидетельствуют о сохранении интереса к рынку цифровых валют  среди клиентов финансовых организаций, поднадзорных Банку России  (далее – финансовые организации).   Ранее Банк России неоднократно отмечал, что распространение цифровых валют в Российской Федерации сопряжено с высокими рисками  для благосостояния российских граждан и стабильности финансовой   системы, а также рисками вовлечения клиентов финансовых орг

In [97]:
import numpy as np

def get_detailed_instruct(query: str, task_description: str = 'Given a web search query, retrieve relevant passages that answer the query') -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

class Encoder:
    def __init__(self, text_by_pages: dict[int,str], document_name: str = "", chunk_size: int = 1) -> None:
        self.text_by_pages = text_by_pages
        self.document_name = document_name
        self.chunk_size = chunk_size
        self.encoded_text_by_pages = text_by_pages.copy()
        self.model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

    def encode_document(self):
        for page_key in self.text_by_pages:
            chunk = ""
            for i in range(self.chunk_size):
                chunk += self.text_by_pages.get(page_key + i, "")
            embedding = self.model.encode(chunk, normalize_embeddings=True)
            self.encoded_text_by_pages[page_key] = embedding
        return self.encoded_text_by_pages

    def encode_query(self, query: str):
        return self.model.encode(get_detailed_instruct(query))


class VectorCalculator:
    def __init__(self, encoded_query, encoded_document_by_chunks: dict[int, str]) -> None:
        self.encoded_query = encoded_query
        self.encoded_document_by_chunks = encoded_document_by_chunks
        self.scores = []

    def get_score_by_chunk(self, chunk_num):
        vec = self.encoded_document_by_chunks[chunk_num]
        return vec @ self.encoded_query / np.linalg.norm(vec)

    def get_scores(self):
        self.scores = [self.get_score_by_chunk(chunk_num) for chunk_num in self.encoded_document_by_chunks]
        return self.scores


class RAG:
    pass

In [ ]:
# FOR SANYA

urls = []

for url in urls:
  doc = DocumentHolder(url)
  encoder = Encoder(doc.text)
  query_embedded = encoder.encode_query("Совет директоров Банка России принял решение от 26 12 2023, какое?")
  chunks_embedded = encoder.encode_document()
  vec = VectorCalculator(query_embedded, chunks_embedded)



In [ ]:
doc = DocumentHolder("https://www.cbr.ru/Crosscut/LawActs/File/7668")
encoder = Encoder(doc.text)


In [102]:
vec = VectorCalculator(encoder.encode_query("Совет директоров Банка России принял решение от 26 12 2023, какое?"),encoder.encode_document())

In [103]:
vec.get_scores()

In [109]:
encoder.text_by_pages[]

'3 предоставлением информации указанным третьим лицам, а также учитывать необходимость обеспечения сохранности предоставленной информации (например, путем заключения соглашений о неразглашении конфиденциальной информации).  Также в целях снижения риска введения мер ограничительного характера КО (ГКО БГ) рекомендуется, придерживаясь принципа осмотрительности, не раскрывать в МСФО-отчетности чувствительные сведения (сведения, раскрытие которых сопряжено (потенциально сопряжено) с риском введения мер ограничительного характера в отношении КО и (или) иных лиц), не подлежащие раскрытию в РСБУ-отчетности в соответствии с приложением 2 к Решению. Головным организациям банковских холдингов (управляющим компаниям банковских холдингов – в случае возложения на них обязанностей головных организаций банковских холдингов) при подготовке к раскрытию годовой (промежуточной) консолидированной финансовой отчетности рекомендуется учитывать решение Совета директоров Банка России от 26.12.2023, предусматри

In [105]:
vec.encoded_document_by_chunks[3]

array([ 0.01788306,  0.02325751, -0.014547  , ..., -0.00067458,
       -0.0288057 , -0.00957037], dtype=float32)

In [108]:
np.argmax(vec.scores) + 1

2

In [112]:
#np.argsort(vec.scores)[::-1][:5]

array([ 2,  0,  1, 30, 33])

In [ ]:
# get 3 most popular ...

for i in np.argsort(vec.scores)[::-1][:3]:
    print(encoder.text_by_pages[i+1])